In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

# Read the Parquet file
df = spark.read.parquet("working_data/Filtered_Fish_Data.parquet")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/07/13 15:23:39 WARN Utils: Your hostname, codespaces-6557a2, resolves to a loopback address: 127.0.0.1; using 10.0.2.180 instead (on interface eth0)
25/07/13 15:23:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/13 15:23:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df.show(5)
df.printSchema()
df.select("SURVEY_ID").count()

+---------+---------------+----------+------------+--------------------+-----------------+----------------------+---------------------+-----------+--------+
|SURVEY_ID|EVENT_DATE_YEAR|SPECIES_ID|SPECIES_NAME|          LATIN_NAME|SURVEY_RANKED_NGR|SURVEY_RANKED_NORTHING|SURVEY_RANKED_EASTING|FISH_LENGTH|AGE_BAND|
+---------+---------------+----------+------------+--------------------+-----------------+----------------------+---------------------+-----------+--------+
|   111032|           1989|       196|        Rudd|Scardinius erythr...|     SE5530009200|                409200|               455300|        142|     >4+|
|    72230|           2002|       196|        Rudd|Scardinius erythr...|     SE7920071500|                471500|               479200|         29|      0+|
|    51949|           2001|       196|        Rudd|Scardinius erythr...|     SU5880008700|                108700|               458800|        157|      0+|
|    67760|           2001|       196|        Rudd|Scardin

4915478

In [3]:
from pyspark.sql.functions import col

df = df.filter(
    (col("AGE_BAND").isNotNull()) &
    (~col("AGE_BAND").contains(">")) &
    (~col("AGE_BAND").contains("<"))
)
df.show(5)
df.select("SURVEY_ID").count()

+---------+---------------+----------+------------+--------------------+-----------------+----------------------+---------------------+-----------+--------+
|SURVEY_ID|EVENT_DATE_YEAR|SPECIES_ID|SPECIES_NAME|          LATIN_NAME|SURVEY_RANKED_NGR|SURVEY_RANKED_NORTHING|SURVEY_RANKED_EASTING|FISH_LENGTH|AGE_BAND|
+---------+---------------+----------+------------+--------------------+-----------------+----------------------+---------------------+-----------+--------+
|    72230|           2002|       196|        Rudd|Scardinius erythr...|     SE7920071500|                471500|               479200|         29|      0+|
|    51949|           2001|       196|        Rudd|Scardinius erythr...|     SU5880008700|                108700|               458800|        157|      0+|
|    67760|           2001|       196|        Rudd|Scardinius erythr...|     TA0910054200|                454200|               509100|         30|      0+|
|   150298|           2017|       196|        Rudd|Scardin

170861

In [4]:
df = df.filter(
    (col("EVENT_DATE_YEAR") >= 2005)
)
df.show(5)
df.select("SURVEY_ID").count()

+---------+---------------+----------+------------+--------------------+-----------------+----------------------+---------------------+-----------+--------+
|SURVEY_ID|EVENT_DATE_YEAR|SPECIES_ID|SPECIES_NAME|          LATIN_NAME|SURVEY_RANKED_NGR|SURVEY_RANKED_NORTHING|SURVEY_RANKED_EASTING|FISH_LENGTH|AGE_BAND|
+---------+---------------+----------+------------+--------------------+-----------------+----------------------+---------------------+-----------+--------+
|   150298|           2017|       196|        Rudd|Scardinius erythr...|     TG4050021700|                321700|               640500|        143|       3|
|   150298|           2017|       196|        Rudd|Scardinius erythr...|     TG4050021700|                321700|               640500|        118|       3|
|   150298|           2017|       196|        Rudd|Scardinius erythr...|     TG4050021700|                321700|               640500|        136|       3|
|   150298|           2017|       196|        Rudd|Scardin

99241

In [5]:
from pyspark.sql.functions import regexp_replace

df = df.withColumn("AGE_BAND", regexp_replace("AGE_BAND", r"\+", ""))
df.select("AGE_BAND").show(10)
df.select("SURVEY_ID").count()

+--------+
|AGE_BAND|
+--------+
|       3|
|       3|
|       3|
|       3|
|       3|
|       3|
|       3|
|       3|
|       3|
|       3|
+--------+
only showing top 10 rows


99241

In [6]:
from pyspark.sql.functions import when
from pyspark.sql.functions import first

df = df.withColumn(
    "SPECIES_ID",
    when(col("SPECIES_ID") == 212, 213).otherwise(col("SPECIES_ID"))
).withColumn(
    "SPECIES_NAME",
    when(col("SPECIES_NAME") == "Pike varieties", "Pike").otherwise(col("SPECIES_NAME"))
)

species_counts_post = df.groupBy("SPECIES_ID", "SPECIES_NAME", "LATIN_NAME") \
    .count() \
    .orderBy("SPECIES_ID", "SPECIES_NAME", "LATIN_NAME")
species_counts_post.show()

+----------+------------+--------------------+-----+
|SPECIES_ID|SPECIES_NAME|          LATIN_NAME|count|
+----------+------------+--------------------+-----+
|       151|      Barbel|       Barbus barbus| 1401|
|       152|        Chub|  Leuciscus cephalus|22333|
|       153|        Dace| Leuciscus leuciscus|19883|
|       154|    Grayling| Thymallus thymallus| 2337|
|       181|       Bleak|   Alburnus alburnus|  884|
|       182|Common bream|       Abramis brama| 3579|
|       193|     Gudgeon|         Gobio gobio| 1519|
|       194|       Roach|     Rutilus rutilus|42640|
|       196|        Rudd|Scardinius erythr...|  613|
|       199|Silver bream|    Abramis bjoerkna|  474|
|       211|       Perch|   Perca fluviatilis| 1948|
|       213|        Pike|         Esox lucius| 1630|
+----------+------------+--------------------+-----+



In [7]:
from pyspark.sql.functions import expr

# List of quantiles to compute
quantiles = [0.1, 0.4, 0.5, 0.6, 0.9]

# For each species and age, compute the quantiles of FISH_LENGTH
growth_curves = (
    df.groupBy("SPECIES_NAME", "AGE_BAND")
      .agg(
          expr("percentile_approx(FISH_LENGTH, 0.1) as very_slow"),
          expr("percentile_approx(FISH_LENGTH, 0.4) as slow"),
          expr("percentile_approx(FISH_LENGTH, 0.5) as average"),
          expr("percentile_approx(FISH_LENGTH, 0.6) as fast"),
          expr("percentile_approx(FISH_LENGTH, 0.9) as very_fast"),
          expr("count(FISH_LENGTH) as n")
      )
      .orderBy("SPECIES_NAME", expr("cast(AGE_BAND as int)"))
)

growth_curves.show(100, truncate=False)

+------------+--------+---------+----+-------+----+---------+----+
|SPECIES_NAME|AGE_BAND|very_slow|slow|average|fast|very_fast|n   |
+------------+--------+---------+----+-------+----+---------+----+
|Barbel      |0       |30       |36  |39     |44  |105      |19  |
|Barbel      |1       |71       |96  |102    |107 |126      |150 |
|Barbel      |2       |110      |142 |156    |165 |211      |237 |
|Barbel      |3       |163      |193 |213    |222 |262      |177 |
|Barbel      |4       |210      |247 |268    |276 |350      |78  |
|Barbel      |5       |252      |310 |329    |350 |404      |78  |
|Barbel      |6       |325      |390 |415    |450 |505      |62  |
|Barbel      |7       |372      |452 |477    |493 |580      |79  |
|Barbel      |8       |387      |452 |472    |496 |568      |74  |
|Barbel      |9       |444      |495 |510    |527 |582      |87  |
|Barbel      |10      |448      |505 |523    |544 |605      |84  |
|Barbel      |11      |460      |534 |549    |570 |642      |6

In [8]:
growth_curves.coalesce(1).write.mode("overwrite").option("header", True).csv("outputs/growth_curves.csv")

In [9]:
from pyspark.sql.functions import expr, when, col

# Define the centre of England (British National Grid)
centre_northing = 350000

# Add a region column: 'North' if northing >= centre, else 'South'
df_region = df.withColumn(
    "region_ns",
    when(col("SURVEY_RANKED_NORTHING") >= centre_northing, "North").otherwise("South")
)

# For each species, age, and region, compute the quantiles of FISH_LENGTH
growth_curves_region = (
    df_region.groupBy("SPECIES_NAME", "AGE_BAND", "region_ns")
      .agg(
          expr("percentile_approx(FISH_LENGTH, 0.1) as very_slow"),
          expr("percentile_approx(FISH_LENGTH, 0.4) as slow"),
          expr("percentile_approx(FISH_LENGTH, 0.5) as average"),
          expr("percentile_approx(FISH_LENGTH, 0.6) as fast"),
          expr("percentile_approx(FISH_LENGTH, 0.9) as very_fast"),
          expr("count(FISH_LENGTH) as n")
      )
      .orderBy("SPECIES_NAME", expr("cast(AGE_BAND as int)"), "region_ns")
)

growth_curves_region.show(100, truncate=False)

+------------+--------+---------+---------+----+-------+----+---------+----+
|SPECIES_NAME|AGE_BAND|region_ns|very_slow|slow|average|fast|very_fast|n   |
+------------+--------+---------+---------+----+-------+----+---------+----+
|Barbel      |0       |North    |66       |66  |66     |66  |66       |1   |
|Barbel      |0       |South    |30       |36  |38     |44  |105      |18  |
|Barbel      |1       |North    |62       |82  |91     |94  |112      |41  |
|Barbel      |1       |South    |82       |102 |105    |110 |132      |109 |
|Barbel      |2       |North    |113      |129 |132    |137 |195      |70  |
|Barbel      |2       |South    |108      |156 |165    |175 |215      |167 |
|Barbel      |3       |North    |159      |177 |181    |190 |241      |62  |
|Barbel      |3       |South    |165      |215 |223    |229 |272      |115 |
|Barbel      |4       |North    |214      |247 |263    |272 |360      |28  |
|Barbel      |4       |South    |209      |246 |268    |278 |340      |50  |

In [10]:
growth_curves_region.coalesce(1).write.mode("overwrite").option("header", True).csv("outputs/growth_curves_region.csv")

In [11]:
from pyspark.sql.functions import sum as spark_sum

# Aggregate counts for each species by summing 'n' for North and South
species_region_counts = (
    growth_curves_region.groupBy("SPECIES_NAME")
    .agg(
        spark_sum(
            when(col("region_ns") == "North", col("n")).otherwise(0)
        ).alias("north_count"),
        spark_sum(
            when(col("region_ns") == "South", col("n")).otherwise(0)
        ).alias("south_count"),
        spark_sum("n").alias("total_count")
    )
)

species_region_counts.show(truncate=False)

+------------+-----------+-----------+-----------+
|SPECIES_NAME|north_count|south_count|total_count|
+------------+-----------+-----------+-----------+
|Roach       |5692       |36948      |42640      |
|Chub        |3918       |18415      |22333      |
|Bleak       |549        |335        |884        |
|Rudd        |11         |602        |613        |
|Barbel      |491        |910        |1401       |
|Dace        |2871       |17012      |19883      |
|Common bream|277        |3302       |3579       |
|Pike        |784        |846        |1630       |
|Gudgeon     |1351       |168        |1519       |
|Perch       |1245       |703        |1948       |
|Grayling    |1775       |562        |2337       |
|Silver bream|22         |452        |474        |
+------------+-----------+-----------+-----------+



In [13]:
import pandas as pd

# Collect the growth_curves DataFrame to a Pandas DataFrame
growth_curves_pd = growth_curves.toPandas()

# Create a Pandas Excel writer using XlsxWriter as the engine
with pd.ExcelWriter("outputs/growth_curves_by_species.xlsx", engine="xlsxwriter") as writer:
    for species, group in growth_curves_pd.groupby("SPECIES_NAME"):
        group.to_excel(writer, sheet_name=species[:31], index=False)